In [1]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from keras.models import Model
from imutils import paths
import numpy as np
import pickle
import random
import os
import cv2
import time

In [2]:
BASE_PATH = "dataset"

# define the names of the training, testing, and validation
# directories
TRAIN = "training"

# initialize the list of class label names
CLASSES = ["positive", "negative"]
# set the batch size
BATCH_SIZE = 32

# initialize the label encoder file path and the output directory to
# where the extracted features (in CSV file format) will be stored
LE_PATH = os.path.sep.join(["output", "le.cpickle"])
BASE_CSV_PATH = "output"
# set the path to the serialized model after training
MODEL_PATH = os.path.sep.join(["output", "model.cpickle"])


In [3]:
# load the VGG16 network and initialize the label encoder
print("[INFO] loading network...")
vgg16 = VGG16(weights="imagenet", include_top=True)
# Using fc2 layer as our feature vector
featureExtractor = Model(inputs=vgg16.input, outputs=vgg16.get_layer('fc2').output)
le = None

[INFO] loading network...


## Preprocessing and Feature Extraction

In [4]:

# grab all image paths in the current split
print("[INFO] processing '{} split'...".format(TRAIN))
p = os.path.sep.join([BASE_PATH, TRAIN])
imagePaths = list(paths.list_images(p))
# randomly shuffle the image paths and then extract the class
# labels from the file paths
random.shuffle(imagePaths)
labels = [p.split(os.path.sep)[-2] for p in imagePaths]
# if the label encoder is None, create it
if le is None:
    le = LabelEncoder()
    le.fit(labels)
# open the output CSV file for writing
csvPath = os.path.sep.join([BASE_CSV_PATH, "{}.csv".format(TRAIN)])
csv = open(csvPath, "w")
# loop over the images in batches
for (b, i) in enumerate(range(0, len(imagePaths), BATCH_SIZE)):
    # extract the batch of images and labels, then initialize the
    # list of actual images that will be passed through the network
    # for feature extraction
    print("[INFO] processing batch {}/{}".format(b + 1, int(np.ceil(len(imagePaths) / float(BATCH_SIZE)))))
    batchPaths = imagePaths[i:i + BATCH_SIZE]
    batchLabels = le.transform(labels[i:i + BATCH_SIZE])
    batchImages = []
    # loop over the images and labels in the current batch
    for imagePath in batchPaths:
        # load the input image using the Keras helper utility
        # while ensuring the image is resized to 224x224 pixels
        image = load_img(imagePath, target_size=(224, 224))
        image = img_to_array(image)
        # preprocess the image by (1) expanding the dimensions and
        # (2) subtracting the mean RGB pixel intensity from the
        # ImageNet dataset
        image = np.expand_dims(image, axis=0)
        image = preprocess_input(image)
        # add the image to the batch
        batchImages.append(image)
    # pass the images through the network and use the outputs as
    # our actual features, then reshape the features into a
    # flattened volume
    batchImages = np.vstack(batchImages)
    features = featureExtractor.predict(batchImages, batch_size=BATCH_SIZE)
    features = features.reshape((features.shape[0], 4096))
    # loop over the class labels and extracted features
    for (label, vec) in zip(batchLabels, features):
        # construct a row that exists of the class label and
        # extracted features
        vec = ",".join([str(v) for v in vec])
        csv.write("{},{}\n".format(label, vec))
# close the CSV file
csv.close()

# serialize the label encoder to disk
f = open(LE_PATH, "wb")
f.write(pickle.dumps(le))
f.close()

[INFO] processing 'training split'...
[INFO] processing batch 1/41
[INFO] processing batch 2/41
[INFO] processing batch 3/41
[INFO] processing batch 4/41
[INFO] processing batch 5/41
[INFO] processing batch 6/41
[INFO] processing batch 7/41
[INFO] processing batch 8/41
[INFO] processing batch 9/41
[INFO] processing batch 10/41
[INFO] processing batch 11/41
[INFO] processing batch 12/41
[INFO] processing batch 13/41
[INFO] processing batch 14/41
[INFO] processing batch 15/41
[INFO] processing batch 16/41
[INFO] processing batch 17/41
[INFO] processing batch 18/41
[INFO] processing batch 19/41
[INFO] processing batch 20/41
[INFO] processing batch 21/41
[INFO] processing batch 22/41
[INFO] processing batch 23/41
[INFO] processing batch 24/41
[INFO] processing batch 25/41
[INFO] processing batch 26/41
[INFO] processing batch 27/41
[INFO] processing batch 28/41
[INFO] processing batch 29/41
[INFO] processing batch 30/41
[INFO] processing batch 31/41
[INFO] processing batch 32/41
[INFO] proc

## Training Model 

In [10]:
def load_data_split(splitPath):
    # initialize the data and labels
    data = []
    labels = []
    # loop over the rows in the data split file
    for row in open(splitPath):
        # extract the class label and features from the row
        row = row.strip().split(",")
        label = row[0]
        features = np.array(row[1:], dtype="float")
        # update the data and label lists
        data.append(features)
        labels.append(label)
    # return a tuple of the data and labels
    data = np.array(data)
    labels = np.array(labels)
    return (data, labels)
    

In [11]:
# derive the paths to the training and testing CSV files
trainingPath = os.path.sep.join([BASE_CSV_PATH, "{}.csv".format(TRAIN)])
# load the data from disk
print("[INFO] loading data...")
(X, Y) = load_data_split(trainingPath)
# load the label encoder from disk
le = pickle.loads(open(LE_PATH, "rb").read())

[INFO] loading data...


In [12]:
# SPlitting Training and Testing data
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.3, random_state=42)

In [13]:
# train the model
print("[INFO] training model...")
model = SVC(kernel='linear')
model.fit(trainX, trainY)
# evaluate the model
print("[INFO] evaluating...")
preds = model.predict(testX)
print(classification_report(testY, preds, target_names=le.classes_))

[INFO] training model...
[INFO] evaluating...
              precision    recall  f1-score   support

    negative       0.99      1.00      0.99       290
    positive       0.99      0.97      0.98        99

    accuracy                           0.99       389
   macro avg       0.99      0.98      0.99       389
weighted avg       0.99      0.99      0.99       389



## Prediction

In [44]:
results = []

In [45]:
def sliding_window(image, stepSize, windowSize):
    # slide a window across the image
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            # yield the current window
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [46]:
test_image = 'dataset/testing/IMG_700101_000223_0046_NIR.TIF'

In [47]:
original_image = cv2.imread(test_image)
image = original_image.copy()

In [48]:
(winW, winH) = (100, 100)

In [ ]:
for (x, y, window) in sliding_window(image, stepSize=100, windowSize=(winW, winH)):
    # if the window does not meet our desired window size, ignore it
    if window.shape[0] != winH or window.shape[1] != winW:
        continue
    # THIS IS WHERE YOU WOULD PROCESS YOUR WINDOW, SUCH AS APPLYING A
    # MACHINE LEARNING CLASSIFIER TO CLASSIFY THE CONTENTS OF THE
    # WINDOW
    # since we do not have a classifier, we'll just draw the window
    
    
    clone = window.copy()
    
    # Preprocess Image before Classification
    resized = cv2.resize(clone, (224,224))
    image2 = np.expand_dims(resized, axis=0)
    image3 = preprocess_input(image2)
    features = featureExtractor.predict(image3, batch_size=BATCH_SIZE)
    pred = model.predict(features)[0]
    
    if(pred == '1'):
        results.append([(x, y), (x + winW, y + winH)])
        cv2.rectangle(image, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
#     cv2.imshow("Window", clone)
#     cv2.waitKey(1)
#     time.sleep(0.025)
#     break;

In [ ]:
len(results)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(image)
plt.title('my picture')
plt.show()